# Import & Config


In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# from torch.optim import Adam
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

import lightning as L
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger


import os

# import cv2
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from lookup_dict import lookup_dict, id_to_name

In [8]:
root_path = ""
dataset_path = os.path.join(root_path, "dataset")
images_dataset_path = os.path.join(dataset_path, "imageNet_images")
eeg_dataset_path = os.path.join(dataset_path, "eeg")

# Dataset


##### Dataset_1 (using custom dataset method)


In [295]:
# Merge all image folders to create list of images

import random

dir_list = list(os.walk(images_dataset_path))
# skip any unnecessary folders
start_idx = len(dir_list) - 40
images_list = []
for sub_dir in dir_list[start_idx:]:
    # images_list+=sub_dir[2]
    images_list.extend(sub_dir[2])

images_list = [image_name.replace(".JPEG", "") for image_name in images_list]

In [294]:
# Split images list to 8:1:1

# random.shuffle(images_list)

images_total_size = len(images_list)
train_size = int(images_total_size * 0.8)
val_size = int(images_total_size * 0.1)
test_size = images_total_size - train_size - val_size

train_images = images_list[:train_size]
val_images = images_list[train_size : train_size + val_size]
test_images = images_list[-1 * test_size :]

In [282]:
eeg_dataset_name = "eeg_5_95_std.pth"
eeg_dataset = torch.load(os.path.join(eeg_dataset_path, eeg_dataset_name))

In [224]:
class CustomDataset(Dataset):
    def __init__(self, images_list) -> None:
        super().__init__()
        self.x_data = []
        self.y_data = []
        # self.y_data = [image_name.split("_")[0] for image_name in images_list]

        for eeg_segment in eeg_dataset["dataset"]:
            for image_name in images_list:
                if eeg_dataset["images"][eeg_segment["image"]] == image_name:
                    self.x_data.append(eeg_segment["eeg"][:, 20:460])
                    # all_channel_list = np.array(eeg_segment['eeg'])
                    # self.x_data.append(torch.from_numpy(all_channel_list[:,40:480]))
                    # self.x_data.append(torch.FloatTensor([eeg_sequence[40:480] for eeg_sequence in eeg_segment['eeg']]))
                    class_id = image_name.split("_")[0]
                    self.y_data.append(lookup_dict[class_id])
                    # self.y_data.append(eeg_dataset['labels'][eeg_segment['label']])
                    break

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)

In [228]:
train_dataset = CustomDataset(train_images)
val_dataset = CustomDataset(val_images)
test_dataset = CustomDataset(test_images)

In [293]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

##### Dataset_2 (using splitter method)


In [12]:
class EEGDataset(Dataset):
    def __init__(self, eeg_dataset_file_name="eeg_5_95_std.pth") -> None:
        super().__init__()
        loaded = torch.load(os.path.join(eeg_dataset_path, eeg_dataset_file_name))
        self.data = loaded["dataset"]
        self.labels = loaded["labels"]
        self.images = loaded["images"]
        self.size = len(self.data)

    def __getitem__(self, idx):
        # t() -> transpose
        eeg = self.data[idx]["eeg"].t()
        eeg = eeg[20:460, :]

        label = self.data[idx]["label"]
        return eeg, label

    def __len__(self):
        return self.size

In [13]:
class Splitter(Dataset):
    def __init__(self, dataset, split_name="train") -> None:
        super().__init__()
        self.dataset = dataset

        loaded = torch.load(
            os.path.join(eeg_dataset_path, "block_splits_by_image_all.pth")
        )
        self.target_data_indices = loaded["splits"][0][split_name]
        # filter data that is too short
        self.target_data_indices = [
            i
            for i in self.target_data_indices
            if 450 <= self.dataset.data[i]["eeg"].size(1) <= 600
        ]

        self.size = len(self.target_data_indices)

    def __getitem__(self, idx):
        eeg, label = self.dataset[self.target_data_indices[idx]]
        return eeg, label

    def __len__(self):
        return self.size

In [14]:
dataset = EEGDataset(eeg_dataset_file_name="eeg_5_95_std.pth")
loaders = {
    split: DataLoader(
        Splitter(dataset, split_name=split), batch_size=16, shuffle=True, drop_last=True
    )
    for split in ["train", "val", "test"]
}

# Model 만들기


In [15]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

seed = 1563423

In [ ]:
model_config = {}

In [59]:
# with classifier attached
class FeatureExtractorNN(L.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        # seed_everything(seed,workers=True)

        self.input_size = 128
        self.hidden_size = 128
        self.lstm_layers = 1
        self.out_size = 128

        # self.lstm = nn.LSTM(input_size=128,hidden_size=128,num_layers=128)
        self.lstm = nn.LSTM(
            self.input_size,
            self.hidden_size,
            num_layers=self.lstm_layers,
            batch_first=True,
        )
        self.output = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.out_size),
            nn.ReLU(),
        )
        self.classifer = nn.Sequential(
            nn.Linear(in_features=self.out_size, out_features=40),
            # don't use softmax with cross entropy loss
            # nn.Softmax(dim=1)
        )

        self.loss_fn = nn.CrossEntropyLoss()
        # self.loss_fn = nn.NLLLoss()
        self.training_step_outputs = {"correct_num": 0, "loss_sum": 0}
        self.validation_step_outputs = {"correct_num": 0, "loss_sum": 0}

    def forward(self, input):
        batch_size = input.size(0)
        lstm_init = (
            torch.zeros(self.lstm_layers, batch_size, self.hidden_size),
            torch.zeros(self.lstm_layers, batch_size, self.hidden_size),
        )
        lstm_init = (lstm_init[0].to(device), lstm_init[0].to(device))

        # dont need to transpose because already transposed when creating dataset
        # input = input.transpose(1,2)

        lstm_out, _ = self.lstm(input, lstm_init)
        # tmp_out = lstm_out[:,-1,:] if input.dim()==3 else lstm_out[-1,:]
        tmp_out = lstm_out[:, -1, :]
        out = self.output(tmp_out)
        # print("out shape",out.shape)
        res = self.classifer(out)

        return res

    def configure_optimizers(self):
        # return optim.SGD(self.parameters(),lr=1e-4,weight_decay=0.1)
        # return optim.Adam(self.parameters(),lr=1e-3,weight_decay=0.1)
        # optimizer = optim.Adam(self.parameters(), lr=(1e-4) * 4, weight_decay=0.001)
        optimizer = optim.Adam(self.parameters(), lr=(1e-3))
        # optimizer = optim.SGD(self.parameters(), lr=(1e-4) * 5)
        # self.scheduler = optim.lr_scheduler.LambdaLR(
        #     optimizer, lambda epoch: 0.95**epoch
        # )
        # self.scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=0.0001, max_lr=0.01,step_size_up=5,mode="triangular2")
        self.scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        return [optimizer], [self.scheduler]
        # return [optimizer]

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        out = self(x)
        loss = self.loss_fn(out, y)

        self.log_dict({"train_loss": loss}, prog_bar=True, on_epoch=True)
        preds = out.argmax(dim=1)
        self.training_step_outputs["correct_num"] += (preds == y).sum()
        self.training_step_outputs["loss_sum"] += loss
        return loss

    def on_train_epoch_end(self) -> None:
        num_correct = self.training_step_outputs["correct_num"]
        acc = num_correct / loaders["train"].dataset.__len__()
        loss = self.training_step_outputs["loss_sum"] / loaders["train"].__len__()
        print("\n")
        # print("EPOCH:",self.current_epoch)
        print(
            f"Training accuracy: {acc.item()} ({num_correct.item()}/{loaders['train'].dataset.__len__()} correct)"
        )
        print("Training loss (average):", loss.item())
        print("\n")
        self.training_step_outputs["correct_num"] = 0
        self.training_step_outputs["loss_sum"] = 0

        print("Learning rate:", self.scheduler.get_last_lr())

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        out = self(x)
        loss = self.loss_fn(out, y)

        self.log_dict({"val_loss": loss}, prog_bar=True, on_epoch=True)
        preds = out.argmax(dim=1)
        self.validation_step_outputs["correct_num"] += (preds == y).sum()
        self.validation_step_outputs["loss_sum"] += loss
        # return loss

    def on_validation_epoch_end(self) -> None:
        num_correct = self.validation_step_outputs["correct_num"]
        acc = num_correct / loaders["val"].dataset.__len__()
        loss = self.validation_step_outputs["loss_sum"] / loaders["val"].__len__()
        print("\n")
        # print("EPOCH:",self.current_epoch)
        print(
            f"Validation accuracy: {acc.item()} ({num_correct.item()}/{loaders['val'].dataset.__len__()} correct)"
        )
        print("Validation loss (average):", loss.item())
        print("\n")
        self.validation_step_outputs["correct_num"] = 0
        self.validation_step_outputs["loss_sum"] = 0

    def test_step(self, batch, batch_idx):
        x, y = batch

        out = self(x)
        loss = self.loss_fn(out, y)

        y_hat = torch.argmax(out, dim=1)
        # print("OUT,YHAT:",out,y_hat)
        test_acc = torch.sum(y == y_hat).item() / (len(y) * 1.0)

        self.log_dict(
            {"test_loss": loss, "test_acc": test_acc}, prog_bar=True, on_epoch=True
        )
        # print("   ||   test loss:",loss.item(), "   ||   test accuracy:",test_acc )

# Training


In [41]:
# seed = 1563423
model = FeatureExtractorNN()
model.to(device)
trainer = L.Trainer()
# trainer.validate(model,dataloaders=val_loader)
trainer.validate(model, dataloaders=loaders["val"])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 114.79it/s]

Validation accuracy: 0.017552657052874565 (35/1994 correct)
Validation loss (average): 3.6950571537017822


Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 114.19it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    3.6950571537017822     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 3.6950571537017822}]

In [60]:
version_num = 96
epoch = 9
step = 23920
PATH = os.path.join(
    root_path,
    "lightning_logs",
    "version_" + str(version_num),
    "checkpoints",
    "epoch=" + str(epoch) + "-step=" + str(step) + ".ckpt",
)

model = FeatureExtractorNN()
model.to(device)
# model = FeatureExtractorNN.load_from_checkpoint(PATH)

lr_monitor = LearningRateMonitor(logging_interval="epoch")
logger = TensorBoardLogger(
    "/Users/ms/cs/ML/NeuroImagen/lightning_logs",
    name="Adam_1-e3_StepLR_10_0.5",
    version=None,
)

trainer = L.Trainer(max_epochs=200, callbacks=[lr_monitor], logger=logger)
# trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader,ckpt_path=PATH)
trainer.validate(model, dataloaders=loaders["val"])
trainer.fit(model, train_dataloaders=loaders["train"], val_dataloaders=loaders["val"])
# trainer.fit(model,train_dataloaders=train_loader)
# trainer.validate(model, dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/ms/cs/ML/NeuroImagen/lightning_logs/Adam_1-e3_StepLR_10_0.5


Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 108.35it/s]

Validation accuracy: 0.020561685785651207 (41/1994 correct)
Validation loss (average): 3.6930313110351562


Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 107.47it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    3.6930313110351562     │
└───────────────────────────┴───────────────────────────┘


  | Name      | Type             | Params
-----------------------------------------------
0 | lstm      | LSTM             | 132 K 
1 | output    | Sequential       | 16.5 K
2 | classifer | Sequential       | 5.2 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
153 K     Trainable params
0         Non-trainable params
153 K     Total params
0.615     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 92.73it/s]

Validation accuracy: 0.001003009034320712 (2/1994 correct)
Validation loss (average): 0.05971241369843483


Epoch 0: 100%|██████████| 497/497 [00:15<00:00, 31.50it/s, v_num=0, train_loss_step=3.400]

Validation accuracy: 0.03460381180047989 (69/1994 correct)
Validation loss (average): 3.5838534832000732


Epoch 0: 100%|██████████| 497/497 [00:17<00:00, 28.70it/s, v_num=0, train_loss_step=3.400, val_loss=3.580, train_loss_epoch=3.650]

Training accuracy: 0.036688026040792465 (292/7959 correct)
Training loss (average): 3.6482152938842773


Learning rate: [0.001]
Epoch 1: 100%|██████████| 497/497 [00:17<00:00, 28.06it/s, v_num=0, train_loss_step=3.140, val_loss=3.580, train_loss_epoch=3.650]

Validation accuracy: 0.054663993418216705 (109/1994 correct)
Validation loss (average): 3.622897148132324


Epoch 1: 100%|██████████| 497/497 [00:19<00:00, 25.75it/s, v_num=0, train_loss_step=3.140, val_loss=3.620, train_los

In [354]:
trainer.validate(model, dataloaders=loaders["val"])

Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 108.29it/s]

Validation accuracy: 0.2096288800239563 (418/1994 correct)
Validation loss (average): 3.610106945037842


Validation DataLoader 0: 100%|██████████| 124/124 [00:01<00:00, 107.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     3.610106945037842     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 3.610106945037842}]

# Testing


In [170]:
trainer.test(model, dataloaders=test_loader)

Testing DataLoader 0:   1%|▏         | 1/76 [00:00<00:01, 61.13it/s]

Testing DataLoader 0: 100%|██████████| 76/76 [00:00<00:00, 89.34it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            0.0            │
│         test_loss         │     3.689608573913574     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 3.689608573913574, 'test_acc': 0.0}]

In [ ]:
idx = 6
query = val_dataset[idx][0].unsqueeze(dim=0)
# print(query.shape)
query.to(device)
pred = model(query)
# print(pred)
pred = torch.argmax(pred, dim=1)
# pred = pred.max(dim=1)
# print(pred)
print("predicted: ", id_to_name[lookup_dict[pred.item()]])
print("answer: ", id_to_name[lookup_dict[val_dataset[idx][1]]])

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at mps:0

In [55]:
# Calculate test accuracy
num_correct = 0
model.to(device)
for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)
    out = model(x)
    y_hat = out.argmax(dim=1)
    # print(y==y_hat)
    num_correct += (y == y_hat).sum()
acc = num_correct / len(test_loader.dataset)
print("Accuracy:", acc.item())

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])

In [ ]:
acc

tensor([0.], device='mps:0')